take the tmp file and get out average expression for each gene in each tissue

In [4]:
import pandas as pd

import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')
from notebook_helper_functions import *

In [5]:
# get outputs from a config file
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/proteincoding_main.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# load in the tissue ids 
tissue_df = load_tissue_df(config)
tissue_ids = list(tissue_df['Tissue'])

In [3]:
expression = pd.read_csv('/home/klawren/oak/pcqtls/data/raw/GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_gene_tpm.gct.gz', sep='\t', skiprows=2)

KeyboardInterrupt: 

In [ ]:
expression_transposed = expression.drop(columns=['transcript_id(s)']).set_index('gene_id').T

In [ ]:
# load in sample matching
expression_phenotypes = pd.read_csv('/home/klawren/oak/pcqtls/data/raw/GTEx_Analysis_2017-06-05_v8_Annotations_SampleAttributesDS.txt', sep='\t')
expression_phenotypes['tissue_id'] = expression_phenotypes['SMTSD'].str.replace(' - ', '_')
expression_phenotypes['tissue_id'] = expression_phenotypes['tissue_id'].str.replace(' ', '_')
expression_phenotypes['tissue_id'] = expression_phenotypes['tissue_id'].str.replace('(', '')
expression_phenotypes['tissue_id'] = expression_phenotypes['tissue_id'].str.replace(')', '')
expression_phenotypes['sample_id'] = expression_phenotypes['SAMPID'].str.split('-').str[:2].str.join('-')

In [ ]:
# filter expression for each tissue so I don't have to load this whole thing every time
tissue_avg_expression = []
for tissue_id in tissue_ids:
    print(tissue_id)
    tissue_norm_expression = load_expression(config, tissue_id)
    rna_seq_ids = tissue_norm_expression.columns[4:]
    print(len(rna_seq_ids))
    full_sample_ids = expression_phenotypes[(expression_phenotypes['sample_id'].isin(rna_seq_ids)) & (expression_phenotypes['tissue_id']==tissue_id)]['SAMPID'].values
    print(len(full_sample_ids))
    full_sample_ids = full_sample_ids[pd.Series(full_sample_ids).isin(expression_transposed.index.values)]
    print(len(full_sample_ids))

    tissue_expression = expression_transposed.loc[full_sample_ids]
    tissue_avg_expression.append(tissue_expression.mean(axis=0))

In [ ]:
avg_expression_df = pd.DataFrame(tissue_avg_expression, index=pd.Series(tissue_ids,name='tissue_id'))
avg_expression_df.to_csv(f'{prefix}/data/processed/GTEx_Analysis_RSEMv1.gene_tpm.tissue_avg.csv', sep='\t')